In [1]:
import sqlite3 as sql

import numpy as np
import pandas as pd
from pandas import cut, read_sql
import pickle as pkl
from random import choices
from sklearn import metrics
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import scipy.optimize as optimize

In [2]:
indir_lvl7 = "/groups/icecube/petersen/GraphNetDatabaseRepository/genie_level7_v02.00_neutrinos/merged_db/osc_next_level7_v2.00_genie_120000_140000_160000_truth_and_retro.db"
indir_New_muon_db = "/groups/icecube/petersen/GraphNetDatabaseRepository/osc_next_database_new_muons_peter/Merged_db/osc_next_level3_v2.00_genie_muongun_noise_120000_140000_160000_139008_888003_retro.db"

#Load in truth data
with sql.connect(indir_lvl7) as con:
    query = """
    SELECT
        event_no,L7_oscNext_bool, EventID, RunID, SubEventID, SubrunID
    FROM 
        truth
    """
    truth_lvl7 = read_sql(query,con)
#azimuth_retro, azimuth_sigma, cascade_energy_retro, energy_retro, energy_sigma, event_no, osc_weight, position_x_retro, position_x_sigma, position_y_retro, position_y_sigma, position_z_retro, position_z_sigma, time_retro, time_sigma, track_energy_retro, track_length_retro, zenith_retro, zenith_sigma
with sql.connect(indir_lvl7) as con:
    query = """
    SELECT
        *
    FROM 
        retro
    """
    retro = read_sql(query,con)

with sql.connect(indir_New_muon_db) as con:
    query = """
    SELECT
        event_no, EventID, RunID, SubEventID, SubrunID
    FROM 
        truth
    """
    truth_new_muon = read_sql(query,con)

In [3]:
print(len(truth_lvl7))
print(len(retro))
print(len(truth_new_muon))

merged_lvl7 = pd.merge(truth_lvl7.reset_index(drop=True), retro.reset_index(drop=True), on='event_no',how='inner')
print(len(merged_lvl7))
print(merged_lvl7.head(1))

merged_lvl7.rename(columns={"event_no": "event_no_old"},inplace=True)

retro_with_good_event_no = pd.merge(merged_lvl7.reset_index(drop=True), truth_new_muon.reset_index(drop=True), on=['RunID', 'SubrunID', 'EventID', 'SubEventID'],how='inner')
print(len(retro_with_good_event_no))
print(retro_with_good_event_no.head(1))

8131391
8131391
38885781
8131391
   event_no  L7_oscNext_bool  EventID     RunID  SubEventID  SubrunID  \
0         0              1.0     34.0  120000.0         0.0     307.0   

   L4_MuonClassifier_Data_ProbNu  L4_NoiseClassifier_ProbNu  \
0                        0.98801                   0.993759   

   L7_MuonClassifier_FullSky_ProbNu  L7_PIDClassifier_FullSky_ProbTrack  ...  \
0                          1.022882                            0.340917  ...   

   position_y_retro  position_y_sigma  position_z_retro  position_z_sigma  \
0        -71.288994          5.285423       -393.254089          2.754913   

   time_retro  time_sigma  track_energy_retro  track_length_retro  \
0  9815.21582   22.275391            0.875477            3.912292   

   zenith_retro  zenith_sigma  
0      1.567303      0.469186  

[1 rows x 28 columns]
8131391
   event_no_old  L7_oscNext_bool  EventID     RunID  SubEventID  SubrunID  \
0             0              1.0     34.0  120000.0         0.0   

In [4]:
#Drop something
retro_with_good_event_no.drop(columns='event_no_old',inplace=True)
print(np.sum(retro_with_good_event_no['L7_oscNext_bool']))
print(retro_with_good_event_no.columns)

8130763.0
Index(['L7_oscNext_bool', 'EventID', 'RunID', 'SubEventID', 'SubrunID',
       'L4_MuonClassifier_Data_ProbNu', 'L4_NoiseClassifier_ProbNu',
       'L7_MuonClassifier_FullSky_ProbNu',
       'L7_PIDClassifier_FullSky_ProbTrack', 'azimuth_retro', 'azimuth_sigma',
       'cascade_energy_retro', 'energy_retro', 'energy_sigma', 'osc_weight',
       'position_x_retro', 'position_x_sigma', 'position_y_retro',
       'position_y_sigma', 'position_z_retro', 'position_z_sigma',
       'time_retro', 'time_sigma', 'track_energy_retro', 'track_length_retro',
       'zenith_retro', 'zenith_sigma', 'event_no'],
      dtype='object')


In [5]:
output_location = "/groups/icecube/petersen/GraphNetDatabaseRepository/multi_classification_track_cascade_neutrino/using_new_muons_Peter_database/inference/Retro/"
retro_with_good_event_no.to_csv(output_location+'retro_on_new_muon_test_set_2',index=False)

(37294941, 5)
(98212967, 5)


   old_event_no     RunID  SubrunID  EventID  SubEventID  event_no  osc_weight
0             0  888003.0    5450.0      0.0         0.0  55418455    0.003054
1             1  888003.0    5450.0      1.0         0.0  55418456    0.003054
2             2  888003.0    5450.0      3.0         0.0  55418457    0.003054
3             3  888003.0    5450.0      5.0         0.0  55418458    0.003054
4             4  888003.0    5450.0     11.0         0.0  55418459    0.003054
5             5  888003.0    5450.0     20.0         0.0  55418460    0.003054
6             6  888003.0    5450.0     22.0         0.0  55418461    0.003054
7             7  888003.0    5450.0     23.0         0.0  55418462    0.003054
8             8  888003.0    5450.0     26.0         0.0  55418464    0.003054
9             9  888003.0    5450.0     28.0         0.0  55418465    0.003054
55418458    0.003054
Name: osc_weight, dtype: float64


In [16]:
# output_location = "/groups/icecube/petersen/GraphNetDatabaseRepository/multi_classification_track_cascade_neutrino/using_MP_lvl3/inference/event_selections/osc_weights.csv"
# merged_df.to_csv(output_location,index=False,columns=('event_no','osc_weight'))